# Terceiro trabalho

<b>Aluno: Felipe Takasawa Paiva </b>
<br>
<b>DRE: 120017769</b>

Repositorio do github: (https://github.com/FTPaiva/trab3-datamining)

### 1 - Carregando os dados

Os dados de GPS dos ônibus estavam em JSON, de hora em hora, do dia 25/04/2024 a 10/05/2024. 
Os arquivos de teste são do dia 16/05/2024 a 20/05/2024.
Carreguei os dados para um BD Postgresql. Para isso, alguns filtros foram usados para selecionar apenas os dados úteis, diminuindo o tempo de interação com o banco:
- Selecionei apenas a linhas que seriam usadas na previsão (48 linhas).
- Para carregar o banco, transformei os arquivos JSON em planilhas CSV e usei o método COPY do PostgreSQL, diminuindo consideravelmente o tempo de inserção.

O código para essa parte do trabalho está no notebook JSONtoDB.ipynb (no repositorio).

### 2 - Encontrando as garagens

Para melhor traçar as rotas, precisamos eliminar as idas, voltas e estadias de onibus nas garagens.<br><br>
Para encontrar estes pontos, utilizei apenas os dados onde os ônibus estavam com velocidade = 0 entre 23h-4h. Após coletar estes dados do banco de dados, criei clusters com um raio de 30 metros. 
Obtive o ponto central de cada cluster e incluí em um tabela no banco de dados.

O código para essa parte do trabalho esta em encontrar_garagens.ipynb.

![Pontos finais](garages.jpg)

### 3 - Identificação dos pontos iniciais e finais para cada linha de ônibus

Para identificar os pontos finais e iniciais para cada linha de ônibus, selecionei todos os dados do GPS no banco de dados que estavam fora de um raio de 200 metros de qualquer ponto de garagem identificado anteriormente. Para cada intervalo de 10 minutos, se o ônibus permaneceu em um raio de 10 metros, estes pontos na janela de 10 minutos são considerados como candidatos a pontos finais.  Coloquei entao, os 2 maiores clusters no mapa

O código para essa parte do trabalho esta em encontrar_inicio_final.ipynb.

![Pontos finais](stops.jpg)

### 4 - Identificação dos trajetos dos ônibus

Com os pontos finais de cada linha mapeados, foi possível determinar os trajetos de ida e volta dos ônibus. A estrutura para o cálculo dos trajetos de cada linha foi a seguinte:
- Decidi dividir os trajetos entre "WEEKDAY", "SABADO" e "DOMINGO", pois há alteracoes detrajetos aos fins de semana devido ao fechamento de vias, etc.
- Para cada grupo de dias, selecionei do banco de dados as posições de GPS dos dias correspondentes entre 8h e 23h. Depois ordenei as posições por identificador do ônibus e datahora.
- Ao iterar sobre o dataframe, buscava primeiro um ponto que estive em um raio de 20m de um dos pontos finais da linha.
- A partir desse ponto criei uma linestring com todos os pontos até que este mesmo ônibus chegasse ao raio de 20m outro ponto da linha. Assim, obtive um trajeto válido, desconsiderando casos em que houve um salto muito grande de tempo ou distância entre dois pontos consecutivos para um mesmo ônibus.
- Com todos os trajetos identificados, selecionei como o melhor trajeto aquele que possui a distância total do trajeto mais próxima da mediana das distâncias dos trajetos calculados. Essa aproximação forneceu bons resultados, apesar de saber que se conseguisse interpolar mais pontos usando todos os trajetos, conseguria obter uma "LineString" mais precisa para o trajeto.
- Selecionei o melhor trajeto de ida e o de volta e coloquei no banco de dados, para cada linha e cada conjunto de dias. Portanto, cada linha possui 6 trajetos diferentes.
Abaixo, uma representação gráfica de todos os trajetos de "WEEKDAY" para as linhas de ônibus analisadas.

O código para essa parte do trabalho esta em trajetorias.ipynb.

![Rotas de ônibus](trajectory.png)

### 5 - Previsões

Não fui capaz de terminar essa parte do trabalho a tempo